<a href="https://colab.research.google.com/github/abSETEd/interface_planilha/blob/main/checkwgmaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas bs4 openpyxl
!pip install odfpy
!pip install pyexcel_ods3
!pip install googlemaps
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import odf  # Para ODS com odfpy
from google.colab import files  # Para o upload no Colab
import io  # Para processar o arquivo enviado
import re  # Para expressões regulares
import os  # Pra manipular nomes de arquivo

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='odf')
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        colunas_esperadas = ["Site", "qtd CNPJs", "qtd UF MVX", "CNPJ", "CNPJRaiz", "RazaoSocial", "Logradouro",
                            "Numero", "Complemento", "Bairro", "Cidade", "Uf", "Cep", "CapitalSocial",
                            "Cnae", "CodigoPredio", "NaturezaJuridica", "DescNaturezaJuridica",
                            "Mailing", "SDR", "Data"]
        colunas_faltantes = [col for col in colunas_esperadas if col not in df.columns]
        if colunas_faltantes:
            raise ValueError(f"As colunas {colunas_faltantes} não foram encontradas na planilha.")

        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar o site diretamente
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))

        return telefones_unicos if telefones_unicos else []
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return []

# Função para consultar o Google Maps usando RazaoSocial
def consultar_google_maps(razao_social):
    try:
        query = f"site:maps.google.com {razao_social}"
        busca_url = f"https://www.google.com/maps/place/{query.replace(' ', '+')}"

        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(busca_url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))

        return telefones_unicos if telefones_unicos else []
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar '{razao_social}' no Google Maps: {e}")
        return []

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload no Colab
    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha: {nome_arquivo}")

    telefones = []
    for index, row in df_urls.iterrows():
        url = row['Site']
        razao_social = row['RazaoSocial']
        print(f"\nConsultando: {url} (Razão Social: {razao_social})")

        telefones_site = consultar_site(url)
        if telefones_site:
            print(f"Telefones encontrados no site: {', '.join(telefones_site)}")
        else:
            print("Nenhum telefone no site. Tentando Google Maps com Razão Social...")

        telefones_maps = consultar_google_maps(razao_social) if not telefones_site else []
        if telefones_maps:
            print(f"Telefones encontrados no Google Maps: {', '.join(telefones_maps)}")

        telefones_combinados = list(set(telefones_site + telefones_maps))
        resultado_final = ', '.join(telefones_combinados) if telefones_combinados else "Nenhum telefone encontrado"
        telefones.append(resultado_final)

        print(f"Resultado final para {url}: {resultado_final}")

        time.sleep(2)

    # Primeiro, adiciona a coluna 'Telefone' ao DataFrame original
    df_urls['Telefone'] = telefones

    # Depois, reorganiza as colunas para inserir 'Telefone' entre 'RazaoSocial' e 'Logradouro'
    colunas_antes = df_urls.columns[:df_urls.columns.get_loc('RazaoSocial') + 1].tolist()
    colunas_depois = df_urls.columns[df_urls.columns.get_loc('Logradouro'):].tolist()
    novas_colunas = colunas_antes + ['Telefone'] + colunas_depois

    # Reorganiza o DataFrame com as novas colunas
    df_urls = df_urls[novas_colunas]

    # Solicita o nome do arquivo de saída apenas no final
    nome_saida = input(f"Digite o nome do arquivo de saída (ex.: novos_dados.csv) ou pressione Enter para '{os.path.splitext(nome_arquivo)[0]}_com_telefones.csv': ") or f"{os.path.splitext(nome_arquivo)[0]}_com_telefones.csv"
    df_urls.to_csv(nome_saida, index=False)

    print(f"Nova planilha salva em '{nome_saida}'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install odfpy
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=fcb528b449ec5976274bedadf7cdc41335aee049e720fd2b3261124df372c3bf
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=285827205fe3ea2153378688cf55b15b261df42db4f87667892178e599e190fc
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps
Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


Saving Domínios Potencial- planilha teste 35 (2).ods to Domínios Potencial- planilha teste 35 (2).ods
Carreguei 34 linhas da planilha: Domínios Potencial- planilha teste 35 (2).ods

Consultando: https://DOMINALOG.COM.BR (Razão Social: DOMINALOG EXPRESS LOGISTICA INTEGRADA LTDA)
Erro ao acessar https://DOMINALOG.COM.BR: 406 Client Error: Not Acceptable for url: https://dominalog.com.br/
Nenhum telefone no site. Tentando Google Maps com Razão Social...
Resultado final para https://DOMINALOG.COM.BR: Nenhum telefone encontrado

Consultando: https://BRCONDOS.COM.BR (Razão Social: RESIDENCIAL BOM RETIRO II)
Telefones encontrados no site: (47) 3801.7605
Resultado final para https://BRCONDOS.COM.BR: (47) 3801.7605

Consultando: https://RENAPSI.ORG.BR (Razão Social: REDE NACIONAL DE APRENDIZAGEM, PROMOCAO SOCIAL E INTEGRACAO)
Nenhum telefone no site. Tentando Google Maps com Razão Social...
Resultado final para https://RENAPSI.ORG.BR: Nenhum telefone encontrado

Consultando: https://GRUPOCENE.C